# CONVERT FILES WITH DUCK_DB

In [12]:
import pandas as pd
import duckdb
import os
import time
import numpy as np
import sys
sys.path.insert(0, os.path.abspath(".."))
from utils import time, data_processor, constants

In [13]:
OUTPUT_PATH = r"..\\data_processed\\guo_subs_world\\subs_china\\" 
country = 'CN'

## Ownership history

In [14]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"



def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")

    COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            {year} AS year,

            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            firmographics_guo.postcode AS guo_postcode,
            firmographics_guo.country_iso_code,
            firmographics_guo.city as guo_city,
            firmographics_guo.region_in_country as guo_region_in_country,

            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_sub.type_of_entity AS subsidiary_type_of_entity,
            firmographics_sub.postcode AS subsidiary_postcode,
            firmographics_sub.country_iso_code AS subsidiary_country_iso_code,
            firmographics_sub.city AS subsidiary_city,
            firmographics_sub.region_in_country AS subsidiary_region_in_country,

            key_financials_detailed.number_of_employees as guo_number_of_employees,
            
        FROM 
            '{path}' AS main

        LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials_detailed
        ON 
            main.subsidiary_bvd_id = key_financials_detailed.bvd_id_number
        AND 
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_guo
        ON 
            main.guo_25 = firmographics_guo.bvd_id_number

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."subsidiary_bvd_id" LIKE '{country}%'
        AND
            main."guo_25" NOT LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df


In [15]:
@time.time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)

    # Split the DataFrame into 5 chunks
    chunks = 5
    indices = np.array_split(df.index, chunks)
    slices = [df.iloc[idx] for idx in indices]
    
    # Example of exporting each slice to Stata
    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
    
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")
        

### Test

In [16]:
country = "CN"
year = "2012"
path = "..\\data_raw\\ownership_history\\links_2012\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)

CN - 2012...


In [17]:
df.to_csv("test_china_subs.csv")

In [18]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\\data_processed\\guo_subs_world\\subs_china\\" 

if not os.path.exists(OUTPUT_PATH):
    print(f"Folder does not exist: {OUTPUT_PATH}")
    os.makedirs(OUTPUT_PATH, exist_ok=True)

total_start = time.time()
start_time = time.time()

for country in ['CN']:
    for year in range(2007, 2024):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors/errors_guos_world.json", "w") as f:
    json.dump(errors, f)

CN - 2007...
Data for CN - 2007 - slice_1 has been converted.
Data for CN - 2007 - slice_2 has been converted.
Data for CN - 2007 - slice_3 has been converted.
Data for CN - 2007 - slice_4 has been converted.
Data for CN - 2007 - slice_5 has been converted.
Execution time: 99.4462 seconds
Data for CN - 2007 has been converted to Stata in 99.45 seconds.
CN - 2008...
Data for CN - 2008 - slice_1 has been converted.
Data for CN - 2008 - slice_2 has been converted.
Data for CN - 2008 - slice_3 has been converted.
Data for CN - 2008 - slice_4 has been converted.
Data for CN - 2008 - slice_5 has been converted.
Execution time: 102.5545 seconds
Data for CN - 2008 has been converted to Stata in 202.00 seconds.
CN - 2009...
Data for CN - 2009 - slice_1 has been converted.
Data for CN - 2009 - slice_2 has been converted.
Data for CN - 2009 - slice_3 has been converted.
Data for CN - 2009 - slice_4 has been converted.
Data for CN - 2009 - slice_5 has been converted.
Execution time: 105.1161 secon

In [ ]:
# open errors
import json 

with open("errors/errors_guos_world.json", "r") as f:
    errors = json.load(f)

errors

{}

## Zip the files

In [9]:
ZIP_OUTPOUT = "..\\zipped_files\\subs_china"

data_processor.compress_csv_files(folder_to_zip=OUTPUT_PATH, zip_output=ZIP_OUTPOUT)

Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2023_chunk_4.csv to ..\zipped_files\subs_china\CN_2023_chunk_4.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2014_chunk_3.csv to ..\zipped_files\subs_china\CN_2014_chunk_3.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2014_chunk_5.csv to ..\zipped_files\subs_china\CN_2014_chunk_5.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2014_chunk_1.csv to ..\zipped_files\subs_china\CN_2014_chunk_1.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2010_chunk_5.csv to ..\zipped_files\subs_china\CN_2010_chunk_5.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2017_chunk_1.csv to ..\zipped_files\subs_china\CN_2017_chunk_1.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2022_chunk_2.csv to ..\zipped_files\subs_china\CN_2022_chunk_2.csv.gz
Compressed ..\\data_processed\\guo_subs_world\\subs_china\\CN_2023_chunk_2.c